In [ ]:
import requests
import random
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import time

headers_list = [
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
]

In [ ]:
def setup_selenium_driver():
    firefox_options = Options()
    firefox_options.add_argument('--no-sandbox')
    firefox_options.add_argument('--disable-dev-shm-usage')
    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=firefox_options)
    return driver

In [ ]:
def fetch_html_with_selenium(url):
    driver = setup_selenium_driver()
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "footer")))
    time.sleep(2)  # Wait for any dynamic content to load
    html_content = driver.page_source
    driver.quit()
    return html_content

In [ ]:
def fetch_html_with_requests(url):
    headers = random.choice(headers_list)
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    return response.text

In [ ]:
def parse_html(html):
    soup = bs(html, 'html.parser')
    products = []
    for item in soup.select('.s-result-item'):
        name = item.find('h2').get_text() if item.find('h2') else 'No Name'
        price = item.find('span', class_='a-price') if item.find('span', class_='a-price') else 'No Price'
        products.append({'name': name, 'price': price})
    return products

In [ ]:
def main():
    url = "https://www.amazon.com/s?k=gap+shirts+for+men"
    try:
        html = fetch_html_with_requests(url)
        products = parse_html(html)
        for product in products:
            print(product)
    except requests.HTTPError as e:
        print(f"Requests error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()